In [1]:
import torch
from torch import nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, inital_channel = 7, num_blocks=20, num_classes=15):
        super(ResNet, self).__init__()
        self.in_channels = 256
        self.value_num = 256
        self.policy_num = 2
        self.classes = num_classes**2

        #initial block
        self.conv1 = nn.Conv2d(inital_channel, self.in_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU()

        #RNN Blocks
        self.layers = []
        for _ in range(num_blocks):
            self.layers.append(block(self.in_channels, self.in_channels))
        self.layers = nn.Sequential(*self.layers)

        # Policy Block
        self.policy_conv = nn.Conv2d(self.in_channels, self.policy_num, kernel_size=1, stride=1, padding=1, bias = False)
        self.policy_bn = nn.BatchNorm2d(self.policy_num)
        self.policy_relu = nn.ReLU()
        self.policy_linear = nn.Linear((num_classes+2)**2*self.policy_num, self.classes)  # 수정된 부분

        # Value Block
        self.value_conv = nn.Conv2d(self.in_channels, 1, kernel_size=1, stride=1, padding = 1, bias=False)
        self.value_bn1 = nn.BatchNorm2d(1)
        self.value_linear = nn.Linear((num_classes+2)**2, 1)  # 수정된 부분
        self.value_relu = nn.ReLU()
        self.value_output = nn.Tanh()


    def forward(self, x):
        # Initial block
        out = self.relu(self.bn1(self.conv1(x)))

        # Residual blocks
        out = self.layers(out)

        # Policy head
        policy = self.policy_relu(self.policy_bn(self.policy_conv(out)))
        policy = policy.view(policy.size(0), -1)  # 평탄화
        policy = self.policy_linear(policy)  # 최종 정책 출력

        # Value head
        value = self.value_relu(self.value_bn1(self.value_conv(out)))
        value = value.view(value.size(0), -1)  # 평탄화
        value = self.value_linear(value)
        value = self.value_relu(value)
        value = self.value_output(value)  # 최종 값 출력

        return policy, value

In [2]:
import numpy as np

model = ResNet(ResidualBlock)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_board = np.array([[[0 for _ in range (15)] for _ in range (15)] for _ in range (7)])
board_tensor = torch.tensor(test_board, dtype=torch.float32)


# 모델, 테스트 데이터를 GPU로 이동
board_tensor = board_tensor.to(device)
board_tensor = board_tensor.unsqueeze(0) 
model.to(device)

ResNet(
  (conv1): Conv2d(7, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (layers): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1,

In [3]:
from torchsummary import summary
summary(model, input_size=(7, 15, 15))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 256, 15, 15]          16,128
       BatchNorm2d-2          [-1, 256, 15, 15]             512
              ReLU-3          [-1, 256, 15, 15]               0
            Conv2d-4          [-1, 256, 15, 15]         589,824
       BatchNorm2d-5          [-1, 256, 15, 15]             512
              ReLU-6          [-1, 256, 15, 15]               0
            Conv2d-7          [-1, 256, 15, 15]         589,824
       BatchNorm2d-8          [-1, 256, 15, 15]             512
              ReLU-9          [-1, 256, 15, 15]               0
    ResidualBlock-10          [-1, 256, 15, 15]               0
           Conv2d-11          [-1, 256, 15, 15]         589,824
      BatchNorm2d-12          [-1, 256, 15, 15]             512
             ReLU-13          [-1, 256, 15, 15]               0
           Conv2d-14          [-1, 256,

In [4]:
# 모델의 첫 번째 파라미터가 위치한 디바이스 확인
model_device = next(model.parameters()).device
print(model_device)

# board_tensor의 디바이스 확인
print(board_tensor.device)


cuda:0
cuda:0


In [5]:
model.eval()
with torch.no_grad():  # Gradient 계산 비활성화
    policy, value = model(board_tensor)

In [10]:
print(policy)
print(len(policy[0]))

tensor([[ 0.0218, -0.0533,  0.0157,  0.0021,  0.0575, -0.0318,  0.0292, -0.0822,
          0.0153, -0.0642, -0.0719,  0.0244,  0.0481, -0.0174,  0.0006, -0.0448,
         -0.0305,  0.0359, -0.1112, -0.0356,  0.0637,  0.0625, -0.0367, -0.0009,
          0.1019,  0.0459,  0.0156, -0.0762, -0.0582,  0.0782, -0.1073,  0.0592,
          0.0390,  0.0531, -0.0113, -0.0924, -0.0076, -0.0262,  0.0036,  0.0226,
         -0.0682, -0.0384, -0.0095,  0.0149, -0.0104, -0.0284, -0.0607, -0.0672,
          0.0714,  0.0320, -0.0318,  0.0058, -0.0715, -0.1108, -0.0430,  0.0756,
         -0.0029,  0.1285,  0.0453, -0.0646, -0.0042,  0.0171,  0.0779,  0.0306,
          0.1207,  0.0206,  0.0726,  0.0138,  0.0275,  0.0046, -0.0503,  0.0034,
         -0.0197,  0.0232, -0.0279,  0.0500, -0.0624,  0.0471, -0.0226, -0.0258,
         -0.0821,  0.0139,  0.0385, -0.0222,  0.0596,  0.0034, -0.1289,  0.0930,
         -0.0383,  0.0373, -0.1704, -0.1518,  0.0258,  0.0413, -0.0725, -0.0049,
          0.0502, -0.0010,  

In [11]:
print(value)
print(len(value[0]))

tensor([[0.0219]], device='cuda:0')
1
